In [1]:
import pandas as pd
import pytest

# yield構文

## 概要

- 関数を一時的に実行停止させることができる機能がある。つまりジェネレータの一種といえる。
- 基本的には関数内部で使われる。
- メリットとしては、関数の小規模なコンポーネント化・メモリ削減・単体テストを実行しやすい点にありそう。

## 作業項目

- 最も基本的な実装例
- pandas.dataframeを使った実行例
- 単体テストでの実行例



In [2]:
def myfunc():
    yield 0
    yield 1
    yield 2

# 呼び出す順番に応じて、値を返す
for x in myfunc():
    print(x)


0
1
2


In [3]:
# 変数化する
x = myfunc()

In [4]:
x.__next__()

0

In [5]:
x.__next__()

1

In [6]:
# next関数でも可能
next(x)


2

In [7]:
# yieldの使用回数を超えたためエラーとなる
x.__next__()


StopIteration: ignored

In [8]:
def myfunc2(fn):
    '''yieldを持つ関数を内包してみる

    Args:
        fn: yieldの関数
    '''
    # x = fn()
    for x_ in fn():
        print(x)
    yield 4

x = myfunc2(myfunc)

In [9]:
# myfunc2 におけるyieldだけ帰ってきた
x.__next__()

<generator object myfunc2 at 0x7ff9c418b9d0>
<generator object myfunc2 at 0x7ff9c418b9d0>
<generator object myfunc2 at 0x7ff9c418b9d0>


4

In [10]:
# pandas.dataframeを利用した実装
@pytest.fixture
def join_a_to_b(a: pd.DataFrame, b: pd.DataFrame, primary_key: str) -> pd.DataFrame:
    '''pandas.dataframeでの途中テーブルにおけるyield文の確認関数

    Args:
        a: pd.DataFrame
        b: pd.DataFrame
        primary_key: str
    Returns:
        pd.DataFrame
    '''
    c = pd.merge(a, b, how='left', on=primary_key)
    yield c
    c.loc[:, 'ダミー'] = ''
    yield c

In [11]:
# ダミーとなるデータフレームの作成
df_a = pd.DataFrame({'id': ['111', '222'], 'values': ['hello', 'world']})
df_b = pd.DataFrame({'id': ['111', '222'], 'data': [100, 200]})
primary_key = 'id'

# 結合関数を実行して変数に代入
df_c = join_a_to_b(df_a, df_b, primary_key)


In [12]:
df_c.__next__()

,id,values,data
0,111,hello,100
1,222,world,200


In [13]:
# 結合された結果に「ダミー」の列が追加されている
# 上でのyieldまでの処理を引き継いだ状態で、実行を続けることが確認できる。
df_c.__next__()

,id,values,data,ダミー
0,111,hello,100,
1,222,world,200,


In [14]:
# 当然エラーとなる
df_c.__next__()

StopIteration: ignored

In [15]:
# pytestは使わず、assert構文を使って自力で単体テストを実装
# google.colabを使っているためファイル単位で管理が難しいため。（※google.driveを用いれば、できないことはない。）
# import pytest 
# pytest --cov -s tests/unit/
def test_join_a_to_b():
    # Arrange
    df_a = pd.DataFrame({
        'id': ['333', '444'], 
        'values': ['train', 'test'], 
    })
    df_b = pd.DataFrame({
        'id': ['333', '444'],
        'data': [300, 400]
    })
    primary_key = 'id'
    excepted_num_columns_for_c_1 = 3
    excepted_num_columns_for_c_2 = 4

    print_1 = 'c_1'
    print_2 = 'c_2'

    # Act
    fn = join_a_to_b(df_a, df_b, primary_key)
    # Assert
    try:
        # fn関数内部のyieldごとの処理結果をfor文で回して、あっているか確認する
        for df_c, print_c, num_column in zip(fn, [print_1, print_2], [excepted_num_columns_for_c_1, excepted_num_columns_for_c_2]):
            # display(df_c)
            print(f'Try to {print_c}')
            assert primary_key in df_c.columns.tolist()
            assert num_column == df_c.shape[1]
            print('OK!')
            yield
    except:
        raise NotImplementedError 


In [16]:
test_res = test_join_a_to_b()

In [17]:
test_res.__next__()

Try to c_1
OK!


In [18]:
# join_a_to_b関数の中間テーブルを確認しながら、段階に応じてテストすることができた。
test_res.__next__()

Try to c_2
OK!
